In [1]:
import time, sys
import json
import random
from PIL import Image
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim, BatchNorm2d
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [2]:
state_dict = {}
file = open("weight_bias_model.txt", "r")

conv1_weights = file.readline()
conv1_bias = file.readline()

conv2_weights = file.readline()
conv2_bias = file.readline()

conv3_weights = file.readline()
conv3_bias = file.readline()

fc1_weights = file.readline()
fc1_bias = file.readline()

fc2_weights = file.readline()
fc2_bias = file.readline()

fc3_weights = file.readline()
fc3_bias = file.readline()

In [3]:
def set_weight_bias_format(data, shape):
    data = data.split(" ")
    data = list(map(float, data))
    return np.array(data).reshape(shape)


In [4]:
conv1_weights = set_weight_bias_format(conv1_weights, (16, 3, 5, 5))
conv1_bias = set_weight_bias_format(conv1_bias, 16)

conv2_weights = set_weight_bias_format(conv2_weights, (32, 16, 5, 5))
conv2_bias = set_weight_bias_format(conv2_bias, (32))

conv3_weights = set_weight_bias_format(conv3_weights, (64, 32, 3, 3))
conv3_bias = set_weight_bias_format(conv3_bias, (64))

fc1_weights = set_weight_bias_format(fc1_weights, (500, 2304))
fc1_bias = set_weight_bias_format(fc1_bias, (500))

fc2_weights = set_weight_bias_format(fc2_weights, (50, 500))
fc2_bias = set_weight_bias_format(fc2_bias, (50))

fc3_weights = set_weight_bias_format(fc3_weights, (2, 50))
fc3_bias = set_weight_bias_format(fc3_bias, (2))

In [5]:
state_dict["conv1.weight"] = Tensor(conv1_weights.astype(np.float32))
state_dict["conv1.bias"] = Tensor(conv1_bias.astype(np.float32))

state_dict["conv2.weight"] = Tensor(conv2_weights.astype(np.float32))
state_dict["conv2.bias"] = Tensor(conv2_bias.astype(np.float32))

state_dict["conv3.weight"] = Tensor(conv3_weights.astype(np.float32))
state_dict["conv3.bias"] = Tensor(conv3_bias.astype(np.float32))

state_dict["fc1.weight"] = Tensor(fc1_weights.astype(np.float32))
state_dict["fc1.bias"] = Tensor(fc1_bias.astype(np.float32))

state_dict["fc2.weight"] = Tensor(fc2_weights.astype(np.float32))
state_dict["fc2.bias"] = Tensor(fc2_bias.astype(np.float32))

state_dict["fc3.weight"] = Tensor(fc3_weights.astype(np.float32))
state_dict["fc3.bias"] = Tensor(fc3_bias.astype(np.float32))

In [6]:
def list_to_string_val(flatten_list):
  return " ".join(str(item) for item in flatten_list)

def float32_to_int(num):
    scaled_num = int(round(num * 1000))
    return scaled_num

def extract_feature_map(x):
  feature_map = x.numpy().flatten()
  feature_map_string_values = list_to_string_val(feature_map)

  with open('feature_map_test.txt','wb') as f:
          f.write(str.encode(feature_map_string_values))
          f.write(str.encode("\n"))

In [7]:
class CustomTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)

    def forward(self, x):
        x = x.reshape((x.shape[0], 3, 224, 224))

        x = self.conv1(x)
        x = x.relu()
        x = x.max_pool2d(2)  
        # extract_feature_map(x)

        x = self.conv2(x)
        x = x.relu()
        x = x.max_pool2d(2)        

        x = self.conv3(x)
        x = x.relu()
        x = x.max_pool2d(2)         

        x = x.reshape(x.shape[0], -1)

        x = self.fc1(x) 
        x = x.relu()

        x = self.fc2(x) 
        x = x.relu()

        x = self.fc3(x)
        return x
        

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias'],
    'fc3.weight': state_dict['fc3.weight'],
    'fc3.bias': state_dict['fc3.bias']
}

model = CustomTinyNet()
load_state_dict(model, checkpoints)

ram used:  0.00 GB, fc3.bias                                          : 100%|██████████| 12/12 [00:00<00:00, 289.66it/s]

loaded weights in 50.61 ms, 0.00 GB loaded at 0.10 GB/s


In [8]:
import torchvision.transforms as transforms

def pytorch_preprocessing(image):  
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Apply the necessary transformations
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image)
    image = image.unsqueeze(0) 
    return Tensor(image.numpy())

In [18]:
import time

def inference_model(image):
    image = pytorch_preprocessing(image)
    start_time = time.time()
    output_tensor = model.forward(image) 
    # print("Time Taken: ", time.time()-start_time)

    predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
    # print("Predicted class:", predicted_class)
    return time.time()-start_time

In [10]:

# !wget https://upload.wikimedia.org/wikipedia/commons/d/d0/German_Shepherd_-_DSC_0346_%2810096362833%29.jpg

image = Image.open('Dataset/Dog/4.jpg')
inference_model(image)

Time Taken:  0.02574324607849121
Predicted class: [1]


In [11]:
# !wget https://i.pinimg.com/736x/59/4d/85/594d857129569aa5ea91ae1a696ea730--red-and-white-white-persian-kittens.jpg

image = Image.open('Dataset/Cat/4.jpg')
inference_model(image)

Time Taken:  0.017928123474121094
Predicted class: [0]


In [20]:
img_path = os.listdir("Dataset/Cat/")
time_taken = 0

for img in img_path:
    image = Image.open('Dataset/Cat/' + img)
    
    time_taken += inference_model(image)
    

time_taken/len(img_path)

0.013648333835601807